In [1]:
!pip install pandas --upgrade --quiet
import pandas as pd
data= pd.read_csv("C:/Users/adedo/Desktop/Data_analytics/data/inventory.csv")
pd.options.display.float_format = "{:,.2f}".format
df = data.fillna(0)
print(df)

      ID CLASS  UNIT COST  RECEIVED  ISSUED
0      1    FD   1,900.00    500.00    0.00
1      2    RM   4,250.00    500.00    0.00
2      3    PM      39.00    400.00    0.00
3      4    PM      23.00      0.00   20.00
4      5    PM      37.00      0.00   20.00
..   ...   ...        ...       ...     ...
147  148    FD   1,750.00      0.00   10.00
148  149    FD     914.40      0.00   10.00
149  150    DR     116.27    200.00    0.00
150  151    FD   1,300.00      0.00   10.00
151  152    FD   1,300.00      0.00   10.00

[152 rows x 5 columns]


In [2]:
#Create an amount column and total quantity column
df['amount'] = (df['RECEIVED'] - df['ISSUED']) * df["UNIT COST"]
df['TotalQty'] = df.groupby("CLASS").cumsum().eval("RECEIVED - ISSUED")
print(df)

      ID CLASS  UNIT COST  RECEIVED  ISSUED       amount  TotalQty
0      1    FD   1,900.00    500.00    0.00   950,000.00    500.00
1      2    RM   4,250.00    500.00    0.00 2,125,000.00    500.00
2      3    PM      39.00    400.00    0.00    15,600.00    400.00
3      4    PM      23.00      0.00   20.00      -460.00    380.00
4      5    PM      37.00      0.00   20.00      -740.00    360.00
..   ...   ...        ...       ...     ...          ...       ...
147  148    FD   1,750.00      0.00   10.00   -17,500.00    367.00
148  149    FD     914.40      0.00   10.00    -9,144.00    357.00
149  150    DR     116.27    200.00    0.00    23,254.00    200.00
150  151    FD   1,300.00      0.00   10.00   -13,000.00    347.00
151  152    FD   1,300.00      0.00   10.00   -13,000.00    337.00

[152 rows x 7 columns]


In [3]:
def get_WAC(df):
    # Set the first average price (We need an inception)
    df.loc[df.index[0], "WAC"] = df.loc[df.index[0], "UNIT COST"]

    # loop over the remaining rows.
    for i in range(1, len(df)):
        # if nothing was received, take the previous value
        if df.loc[df.index[i], "RECEIVED"] == 0:
            df.loc[df.index[i], "WAC"] = df.loc[df.index[i - 1], "WAC"]
        # if something was received
        elif df.loc[df.index[i], "RECEIVED"] > 0:
            df.loc[df.index[i], "WAC"] = (
                df.loc[df.index[i], "amount"]
                + ((df.loc[df.index[i - 1], "WAC"]) * (df.loc[df.index[i - 1], "TotalQty"]))
            ) / (df.loc[df.index[i], "TotalQty"])
        elif df.loc[df.index[i], "RECEIVED"] < 0:
            df.loc[df.index[i], "WAC"] = df.loc[df.index[i - 1], "WAC"]
        else:
            df.loc[df.index[i], "WAC"] = 0

    df["WAC"] = df["WAC"].round(1)

    return df


df = df.groupby("CLASS", group_keys=False).apply(get_WAC)
print(df)

      ID CLASS  UNIT COST  RECEIVED  ISSUED       amount  TotalQty      WAC
0      1    FD   1,900.00    500.00    0.00   950,000.00    500.00 1,900.00
1      2    RM   4,250.00    500.00    0.00 2,125,000.00    500.00 4,250.00
2      3    PM      39.00    400.00    0.00    15,600.00    400.00    39.00
3      4    PM      23.00      0.00   20.00      -460.00    380.00    39.00
4      5    PM      37.00      0.00   20.00      -740.00    360.00    39.00
..   ...   ...        ...       ...     ...          ...       ...      ...
147  148    FD   1,750.00      0.00   10.00   -17,500.00    367.00 1,901.30
148  149    FD     914.40      0.00   10.00    -9,144.00    357.00 1,901.30
149  150    DR     116.27    200.00    0.00    23,254.00    200.00   116.30
150  151    FD   1,300.00      0.00   10.00   -13,000.00    347.00 1,901.30
151  152    FD   1,300.00      0.00   10.00   -13,000.00    337.00 1,901.30

[152 rows x 8 columns]


In [4]:
print(df.loc[df["CLASS"] == "FD"])

      ID CLASS  UNIT COST  RECEIVED  ISSUED      amount  TotalQty      WAC
0      1    FD   1,900.00    500.00    0.00  950,000.00    500.00 1,900.00
8      9    FD   1,950.00      0.00  220.00 -429,000.00    280.00 1,900.00
9     10    FD   1,800.00    250.00    0.00  450,000.00    530.00 1,852.80
10    11    FD   2,000.00      0.00  100.00 -200,000.00    430.00 1,852.80
11    12    FD   2,000.00     77.00    0.00  154,000.00    507.00 1,875.20
12    13    FD   1,850.00      0.00   20.00  -37,000.00    487.00 1,875.20
13    14    FD   2,000.00      0.00   20.00  -40,000.00    467.00 1,875.20
14    15    FD   1,950.00    250.00    0.00  487,500.00    717.00 1,901.30
75    76    FD     757.70      0.00   20.00  -15,154.00    697.00 1,901.30
76    77    FD     757.70      0.00   20.00  -15,154.00    677.00 1,901.30
77    78    FD     757.70      0.00   20.00  -15,154.00    657.00 1,901.30
98    99    FD   1,054.50      0.00   10.00  -10,545.00    647.00 1,901.30
99   100    FD   1,400.00